<a href="https://colab.research.google.com/github/vksepm/hands-on-agentic-ai-building-ai-agents-with-llamaindex-3962125/blob/main/code_03_XX%20Summarization%20with%20reflection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU python-dotenv
!pip install -qU llama-index
!pip install -qU llama-index-llms-openai
!pip install -qU llama-index-embeddings-openai
!pip install -qU llama-index-llms-azure-openai
!pip install -qU llama-index-embeddings-azure-openai
!pip install -qU llama-index-agent-introspective
!pip install -qU llama-index-program-openai
!pip install -qU llama-index-readers-file
!pip install -qU pydantic
!pip install -qU llama-index-core
!pip install -qU llama-index-utils-workflow
!pip install -qU llama-index-tools-wikipedia
!pip install -qU llama-index-readers-json
!pip install -qU llama-index-readers-file
!pip install -qU pymupdf
!pip install -qU llama-index-tools-google
!pip install -qU nbformat
!pip install -qU arize-phoenix
!pip install -qU llama-index
!pip install -qU llama-index-callbacks-arize-phoenix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.3/251.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━

In [2]:
%%bash
git clone https://github.com/vksepm/hands-on-agentic-ai-building-ai-agents-with-llamaindex-3962125.git
mv ./hands-on-agentic-ai-building-ai-agents-with-llamaindex-3962125/* .
rm -rf ./hands-on-agentic-ai-building-ai-agents-with-llamaindex-3962125/

Cloning into 'hands-on-agentic-ai-building-ai-agents-with-llamaindex-3962125'...


### Set up LLM

In [8]:
#Using AzureOpenAI as LLM. Other LLM options are available in LlamaIndex
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import Settings
import os
import nest_asyncio
from google.colab import userdata
from IPython.display import Markdown, display

#Used by LlamaIndex
nest_asyncio.apply()

#Setup the environment variables
os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get('AZURE_OPENAI_ENDPOINT')
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
os.environ["OPENAI_API_VERSION"] = userdata.get('OPENAI_API_VERSION')

#Note : Function calling support only available in GPT-4+
#Create the LLM object
llm=AzureOpenAI(
    model="gpt-4o-mini",
    deployment_name="gpt-4o-mini",
    api_version="2024-08-01-preview",
)

### 03.02. Create introspection agent

In [5]:
from llama_index.agent.introspective import SelfReflectionAgentWorker
from llama_index.agent.introspective import IntrospectiveAgentWorker
from llama_index.core.llms import ChatMessage, MessageRole

#Setup the reflection agent worker
self_reflection_agent_worker = SelfReflectionAgentWorker.from_defaults(
    llm=llm,
    verbose=True,
)

#Setup the introspective agent worker
introspective_agent_worker = IntrospectiveAgentWorker.from_defaults(
    reflective_agent_worker = self_reflection_agent_worker,
    main_agent_worker = None,
    verbose=True,
)

#Create a system prompt defining the function of the agent
system_prompt="""
You are an Product specification summarizer who can summarize a product specification.
For the input provided, create a summary with less than 50 words.

Ensure that the summary focuses on performance specifications
and safety features.
"""
#Instantiate chat history with system message
chat_history = [
    ChatMessage(
        content=system_prompt,
        role=MessageRole.SYSTEM
    )
]

#Create the agent
introspective_agent= introspective_agent_worker.as_agent(
    chat_history=chat_history,
    verbose=True
)

### 03.03. Execute the introspection Agent

In [6]:
from llama_index.readers.file import PyMuPDFReader

#Input file for summarization
loader=PyMuPDFReader()
docs=loader.load(file_path="./datasets/EcoSprint_Specification_Document.pdf")


#Pick the first page of the doc as content
source_content=docs[0].text
print(source_content)

EcoSprint Specification Document
1. Overview
●
The EcoSprint is a revolutionary electric vehicle (EV) designed for efficiency and
performance. With its sleek design and state-of-the-art technology, the EcoSprint
appeals to environmentally conscious drivers who don't want to compromise on style or
driving experience. Ideal for city driving and daily commutes, the EcoSprint offers a
perfect blend of comfort, sustainability, and innovation.
2. Design Specifications
●
Exterior Design: The EcoSprint boasts a modern and aerodynamic silhouette, featuring
smooth lines and a compact form factor. Available in colors like Midnight Black, Ocean
Blue, and Pearl White, it's a head-turner on the road.
●
Interior Design: Inside, the EcoSprint is a realm of comfort and luxury. It offers a
spacious cabin with seating for five, premium upholstery, and customizable ambient
lighting.
3. Performance Specifications
●
Engine and Motor: Powered by a high-efficiency electric motor, the EcoSprint delivers
200 ho

In [7]:
len(docs)

2

In [9]:
response=await introspective_agent.achat(source_content)

# print("\n", response.response)
display(Markdown(f"----\n"))
display(Markdown(f"**Response:** {response.response}"))

> Running step a0008259-dbbe-4dd1-91d9-8df4d27d76f6. Step input: EcoSprint Specification Document
1. Overview
●
The EcoSprint is a revolutionary electric vehicle (EV) designed for efficiency and
performance. With its sleek design and state-of-the-art technology, the EcoSprint
appeals to environmentally conscious drivers who don't want to compromise on style or
driving experience. Ideal for city driving and daily commutes, the EcoSprint offers a
perfect blend of comfort, sustainability, and innovation.
2. Design Specifications
●
Exterior Design: The EcoSprint boasts a modern and aerodynamic silhouette, featuring
smooth lines and a compact form factor. Available in colors like Midnight Black, Ocean
Blue, and Pearl White, it's a head-turner on the road.
●
Interior Design: Inside, the EcoSprint is a realm of comfort and luxury. It offers a
spacious cabin with seating for five, premium upholstery, and customizable ambient
lighting.
3. Performance Specifications
●
Engine and Motor: Powered b

----


**Response:** **EcoSprint Specification Document Summary**

**Overview:** The EcoSprint is an innovative electric vehicle (EV) designed for efficiency and performance, appealing to environmentally conscious drivers. Ideal for city driving and daily commutes, it offers comfort and sustainability.

**Design Specifications:** Features a modern aerodynamic exterior in Midnight Black, Ocean Blue, and Pearl White, with a spacious cabin for five, premium upholstery, and customizable ambient lighting.

**Performance Specifications:** The EcoSprint boasts a high-efficiency electric motor delivering 200 horsepower and 300 Nm of torque, powered by a 50 kWh lithium-ion battery. It offers a range of up to 250 miles on a single charge, accelerates from 0 to 60 mph in 7.3 seconds, and achieves a top speed of 120 mph.

**Technology and Features:** Equipped with a 10-inch touchscreen infotainment system, voice control, adaptive cruise control, lane-keeping assist, automatic emergency braking, and remote monitoring via a smartphone app.

**Safety and Security:** The vehicle includes multiple airbags, a reinforced frame, blind-spot monitoring, remote locking/unlocking, GPS tracking, and intrusion alarms, ensuring comprehensive safety for drivers and passengers.

**Summary:** The EcoSprint combines cutting-edge technology with exceptional performance and safety features, making it a top choice for modern drivers.

Observability

In [ ]:
import os
from google.colab import userdata
import llama_index

PHOENIX_API_KEY = userdata.get('PHOENIX_API_KEY')
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
llama_index.core.set_global_handler(
    "arize_phoenix", endpoint="https://llamatrace.com/v1/traces"
)